In [6]:
%matplotlib inline

import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np

import json
import os
import glob
import datetime

# Essay scoring

## Development set

In [47]:
# Gather all params.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
    
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    prm["Directory"] = os.path.basename(d)
    prm["Time"] = str(datetime.datetime.fromtimestamp(os.stat(d).st_mtime))[:19]
    prm["Fold"] = [i for i in range(5) if os.path.exists(os.path.join(d, "regression_f{}.hdf5".format(i)))]
    prm["Eval"] = [i for i in range(5) if os.path.exists(os.path.join(d, "prediction_f{}.json".format(i)))]

    mse_pool = []
    
    for i in range(5):
        pjs = os.path.join(d, "prediction_f{}.json".format(i))
        
        if os.path.exists(pjs):
            pred = json.load(open(pjs))
            mse_pool += [pred["MSE"]]

    mse_pool = np.array(mse_pool)
    prm["MSEstd"] = mse_pool.std()    
    prm["MSE"] = mse_pool.mean()
    prm["MSEs"] = mse_pool
    
    if os.path.exists(os.path.join(d, "logs_f0.pickle")):
        l = pickle.load(open("{}/logs_f0.pickle".format(d), "rb"))
        df = pd.DataFrame(l["logs_e"])
        prm["R_loss"] = df["loss"].min()
        prm["R_val_loss"] = df["val_loss"].min()

    else:
        prm["R_loss"] = 0
        prm["R_val_loss"] = 0
        
    results += [prm]

cols = "Time R_loss R_val_loss Fold Eval MSE MSEstd MSEs dropout clipnorm emb_fix enc_fix pseq preenc Directory".split()

pd.set_option("display.max_colwidth", 200)
df = pd.DataFrame(results)
df = df[df["pseq"] == "True"]
df[cols].sort_values(by="Time", ascending=False)

/home/naoya-i/py36/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.


,Time,R_loss,R_val_loss,Fold,Eval,MSE,MSEstd,MSEs,dropout,clipnorm,emb_fix,enc_fix,pseq,preenc,Directory
30,2019-01-12 13:05:33,0.011595,0.019982,[0],[],NaN,NaN,[],0.75,10.0,True,False,True,None,e885c38c446af586e887fb790dd90930
9,2019-01-12 12:56:26,0.013358,0.020127,[0],[],NaN,NaN,[],0.75,5.0,True,False,True,None,ed002b2a81be00fd9502b16c1dcf18d3
38,2019-01-12 12:48:49,0.005683,0.020035,[0],[],NaN,NaN,[],0.5,10.0,True,False,True,None,3388e834811fe9c070e5cbf74d02bedf
29,2019-01-12 12:40:15,0.006593,0.020650,[0],[],NaN,NaN,[],0.5,5.0,True,False,True,None,b12e3870579b0c3cf44e991a5707b190
39,2019-01-12 12:33:33,0.004056,0.020989,[0],[],NaN,NaN,[],0.25,10.0,True,False,True,None,afba0cc86df64014a894e262db301ab6
37,2019-01-12 12:24:46,0.004108,0.020731,[0],[],NaN,NaN,[],0.25,5.0,True,False,True,None,967a88ac69959c14fc310b4e727d6e74
26,2019-01-12 11:00:18,0.013032,0.021034,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.207077,0.021028,"[0.17477832335600332, 0.2069103628534164, 0.20235256500706644, 0.21061206024685153, 0.24073340256757852]",0.7,5.0,True,False,True,None,cde3e253163f10bde9887a702811182a
0,2019-01-12 09:42:45,0.017368,0.019604,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.188631,0.018396,"[0.17598836861740502, 0.1777602213648306, 0.1686265442748433, 0.20460608113119388, 0.2161751506085603]",0.7,5.0,True,True,True,None,1df2ac4527c37185cddfbbe2cbb1cf10
7,2019-01-10 08:06:08,0.005481,0.025959,[0],[0],0.264610,0.000000,[0.2646098742678282],0.7,6.0,False,False,True,None,f25284b693ec86cb3c8c6da3a6553cf9
6,2019-01-09 09:50:46,0.018443,0.019998,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.203481,0.019558,"[0.17270962312103763, 0.2207709531401928, 0.18877799411561982, 0.21240548011506646, 0.22274311408665057]",0.7,5.0,True,True,True,output_enc/9780456c95e7c048e2501106fd40c716,46f970b1c6379f85b7ccc1fe68a8af14


## Test set

In [49]:
# Gather all results.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
        
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    
    p = [json.load(open(os.path.join(d, "prediction_f{}.json".format(i)))) for i in range(5) if os.path.exists(os.path.join(d, "prediction_f{}.json".format(i)))]
    p = np.array([x["MSE"] for x in p])
    
    prm["Directory"] = os.path.basename(d)
    prm["MSEs"] = p
    prm["MSE"] = p.mean(), p.std()
    
    if "score_type" not in prm:
        prm["score_type"] = "Organization"
    
    results += [prm]
        
df = pd.DataFrame(results)
df = df.sort_values(by="score_type")
df[["Directory", "MSEs", "MSE", "score_type", "preenc", "enc_fix", "emb_fix", "pretrained", "pseq"]]

/home/naoya-i/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice.
  from ipykernel import kernelapp as app


,Directory,MSEs,MSE,score_type,preenc,enc_fix,emb_fix,pretrained,pseq
26,51a2e8727c1fe1fa27d190f879ce078d,"[0.26496048231386626, 0.23985156913924272, 0.2462020482008063, 0.23528099814833667, 0.26786079507282806]","(0.250831178575016, 0.013216912233143372)",ArgumentStrength,output_enc/c2c4d855a06224fd1096834eed11920d,False,False,False,False
1,5cc10bbc73f490977573fb5b763ef0fc,[],"(nan, nan)",ArgumentStrength,None,False,True,True,False
45,fcbeada6ec3cb1984984fe3ec9cb664e,"[0.2553112997422369, 0.2392242413397571, 0.2528419103068555, 0.2315592930770336, 0.26730756532314587]","(0.24924886195780577, 0.01256338952106033)",ArgumentStrength,output_enc/c2c4d855a06224fd1096834eed11920d,True,True,False,False
22,4d2b32819bf73eecafdc08288f39e34c,"[0.27303764034418093, 0.23817723298100674, 0.25462896249886086, 0.23306868678844408, 0.2626360273298695]","(0.2523097099884724, 0.014910417425571015)",ArgumentStrength,None,False,True,True,False
19,b1809650ecdefed27f07e32a05dc3ade,"[0.2592230096534951, 0.23955059426323458, 0.259447455157723, 0.23680760373543905, 0.267192892992712]","(0.25244431116052074, 0.012027219127379462)",ArgumentStrength,output_enc/9780456c95e7c048e2501106fd40c716,False,False,False,False
15,8adb0fc48cedf9e322e09c406c747e5a,[],"(nan, nan)",ArgumentStrength,None,False,True,True,True
27,cee23dc2ed54aed0911230d84151441e,"[0.25664800333384447, 0.23941677197983893, 0.2723700790054423, 0.23736549721363503, 0.26929889274507046]","(0.2550198488555663, 0.014578783265718775)",ArgumentStrength,output_enc/9780456c95e7c048e2501106fd40c716,True,True,False,False
28,cde3e253163f10bde9887a702811182a,"[0.17477832335600332, 0.2069103628534164, 0.20235256500706644, 0.21061206024685153, 0.24073340256757852]","(0.20707734280618326, 0.021027613516739027)",Organization,None,False,True,True,True
29,2674fd3a66a463d7fd62804995734663,[],"(nan, nan)",Organization,output_enc/750570aed2d16633ecbe4237d2d95b71,False,False,False,True
30,6f27e738794f9eb73bd73c0c868d4cb2,"[0.1722483500131174, 0.1794667317586682, 0.16374400186657234, 0.2143731473086255, 0.21654933758099426]","(0.18927631370559553, 0.021962455607409677)",Organization,output_enc/750570aed2d16633ecbe4237d2d95b71,False,False,False,True


In [ ]:
tn16pn10_pre_sentshuf_finetune = "0cb7a2429b6cc73297413c20570c824f"
tn16pn10_pre_sentshuf_nofinetune = "752aa6160e706a6ffe0f91a1e423b40a"

# Encoder pretraining

In [26]:
# Gather all params.
results = []

for d in glob.glob("../output_enc/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
    
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    prm["Directory"] = os.path.basename(d)
    prm["Time"] = str(datetime.datetime.fromtimestamp(os.stat(d).st_mtime))[:19]
    
    l = pickle.load(open("{}/logs.pickle".format(d), "rb"))
    df = pd.DataFrame(l["logs_e"])
    prm["R_acc"] = df["acc"].max()
    prm["R_val_acc"] = df["val_acc"].max()
    
    results += [prm]

cols = "Time R_acc R_val_acc dropout emb_fix enc_fix shuf Directory".split()

pd.set_option("display.max_colwidth", 50)
df = pd.DataFrame(results)
df[cols].sort_values(by="Time", ascending=False)

,Time,R_acc,R_val_acc,dropout,emb_fix,enc_fix,shuf,Directory
3,2018-12-27 14:18:45,0.993037,0.903384,0.7,False,False,di,a87b827fa7c5151192542ecb2c3af4d2
5,2018-12-27 13:25:00,0.986483,0.825873,0.5,False,False,di,6abbf82fd461ebcd2ac61867427b2a1e
1,2018-12-27 13:02:47,0.996313,0.853712,0.3,False,False,di,da6836f961365a7b348281a00e48bc34
2,2018-12-27 11:37:36,0.992900,0.936681,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=Fa...
4,2018-12-27 11:32:34,0.902512,0.743996,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=Fa...
0,2018-12-27 11:32:27,0.925587,0.742904,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.5_emb_dim=50_emb_fix=Fa...


In [2]:
### Score_normalized
# TN16
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-LSTMdim 300 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding

fold_0: MSE: 0.3643280911204178, MAE: 0.4622013795375824
fold_1: MSE: 0.3197095480935262, MAE: 0.39644437405600474
fold_2: MSE: 0.4364101866079787, MAE: 0.438516518369836   
fold_3: MSE: 0.32989512169457946, MAE: 0.4378773703503965
fold_4: MSE: 0.3643280911204178, MAE: 0.4622013795375824
            
MSE: 0.362

SyntaxError: invalid syntax (<ipython-input-2-e702d4088bd6>, line 3)

In [ ]:
fold0: MSE: 0.19625852776829697 MAE: 0.3492366951704025
fold1: 

In [5]:
# Gather all results.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "prediction_f1.json")):
        continue
        
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    
    p = json.load(open(os.path.join(d, "prediction_f1.json")))
    prm["MSE"] = p["MSE"]
    
    results += [prm]
        
df = pd.DataFrame(results)
df[["MSE", "preenc", "enc_fix", "emb_fix", "pretrained", "pseq"]]

,MSE,preenc,enc_fix,emb_fix,pretrained,pseq
0,0.335248,None,False,True,True,False
1,0.319710,None,False,True,True,False


# Command repo

In [1]:
# Command for training:

# TN16
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 300 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding

# TN16+PN10
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64

# TN16+PN10+pretrain(di. shuffle, fixed)
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --fix-encoder --fix-embedding \
    --pretrained-encoder output_enc/a87b827fa7c5151192542ecb2c3af4d2

# TN16+PN10+pretrain(di. shuffle, not fixed)
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --pretrained-encoder output_enc/a87b827fa7c5151192542ecb2c3af4d2

# TN16+PN10+pretrain(sent. shuffle, fixed)
CUDA_VISIBLE_DEVICES=1 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --fix-encoder --fix-embedding \
    --pretrained-encoder output_enc/clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=False_enc_fix=False_model_type=nea_mot=True_pretrained=False_shuf=sentence

# TN16+PN10+pretrain(sent. shuffle, not fixed)
CUDA_VISIBLE_DEVICES=1 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --pretrained-encoder output_enc/clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=False_enc_fix=False_model_type=nea_mot=True_pretrained=False_shuf=sentence

SyntaxError: invalid syntax (<ipython-input-1-dba85e4206fa>, line 4)

In [ ]:
# Command for sentence encoder pretraining:
CUDA_VISIBLE_DEVICES=1 python src/train_enc.py \
    --model-type nea --dropout 0.3 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --shuffle-type di

CUDA_VISIBLE_DEVICES=1 python src/train_enc.py \
    --model-type nea --dropout 0.3 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --shuffle-type sentence

In [ ]:
# Command for evaluation
CUDA_VISIBLE_DEVICES=1 python src/eval.py \
    --fold 1 \
    --model-dir output/cbc428f99e04b33c2dcb221e7331e07d

In [11]:
import pickle

logs = pickle.load(open("../output/2cfef26bfd48471a2c54726988f72420/logs_f1.pickle", "rb"))
df = pd.DataFrame(logs["logs_e"])
df

,loss,mean_squared_error,val_loss,val_mean_squared_error
0,0.057288,0.057288,0.028863,0.028863
1,0.043255,0.043255,0.035102,0.035102
2,0.041587,0.041587,0.033113,0.033113
3,0.040196,0.040196,0.034093,0.034093
4,0.039479,0.039479,0.027005,0.027005
5,0.038605,0.038605,0.026719,0.026719
6,0.039017,0.039017,0.027171,0.027171
7,0.038305,0.038305,0.026136,0.026136
8,0.037606,0.037606,0.026472,0.026472
9,0.036938,0.036938,0.031719,0.031719


In [12]:
df.describe()

,loss,mean_squared_error,val_loss,val_mean_squared_error
count,35.000000,35.000000,35.000000,35.000000
mean,0.030731,0.030731,0.028449,0.028449
std,0.009068,0.009068,0.002609,0.002609
min,0.016624,0.016624,0.025675,0.025675
25%,0.023180,0.023180,0.026428,0.026428
50%,0.029469,0.029469,0.028146,0.028146
75%,0.037357,0.037357,0.029157,0.029157
max,0.057288,0.057288,0.035102,0.035102
